# Medical Insurance Analysis using Linear Regression

We are provided the medical insurance data in  CSV formatted file. The file contains 7 columns i.e. `age`, `sex`, `bmi`, `children`, `smoker`, `region` and `charges`

Our primary goal in this project is to find the relationship of multiple variables on the cost/`charges` of the insurance.

We will be using the linear regression analysis which finds the best fitting line that describes the relationship between two variables. the line is represented by the equation
> y = mx + b

Where
- **y** represents the **vertical postion** of a datapoint
- **x** represents the **horizontal position** of a datapoint
- **m** represents the **slope** of the line on which this point lies
- **b** represents the **y-intercept** of the line on which this point lies

Let's start our analysis by importing the CSV data and put the data of each column in a separate list.

## Read CSV

In [2]:
import csv

variables = {}
with open('Data/insurance.csv') as insurance_csv:
    insurance_dict = csv.DictReader(insurance_csv)
    for row in insurance_dict:
        for k,v in row.items():
            if k in variables:
                variables[k].append(v)
            else:
                variables[k] = []

#print(variables)

## Slope and y-Intercept calculation

Next we need to write the functions that will find **the slope `m` and y-intercept `b`** of the best fitting line for the `x` and `y` datapoints.

The steps to find **the slope `m` and y-intercept `b`** is as following

1. Calculate the means of x and y:
>x̄ = (x₁ + x₂ + ... + xn) / n

>ȳ = (y₁ + y₂ + ... + yn) / n

where n is the number of data points.

2. Calculate the differences between each data point and the means:
>Δxᵢ = xᵢ - x̄

>Δyᵢ = yᵢ - ȳ

for each data point (xᵢ, yᵢ).

3. Calculate the sum of the products of the differences:
>Σ(Δxᵢ * Δyᵢ)

4. Calculate the sum of the squared differences of x:
>Σ(Δxᵢ²)

5. Calculate the slope (m):
>m = Σ(Δxᵢ * Δyᵢ) / Σ(Δxᵢ²)

6. Calculate the y-intercept (b):
>b = ȳ - m * x̄

In [3]:
def get_mean(list):
    sum = 0
    for i in list:
        sum += float(i)
        
    return float(sum/len(list))

def get_diff_from_mean(list):
    mean = get_mean(list)
    diffs = []
    for i in list:
        diffs.append(float(i) - mean)
    return diffs

def calc_slope(xList, yList):
    delta_x = get_diff_from_mean(xList)
    delta_y = get_diff_from_mean(yList)
    sum_deltax_deltay = 0
    sum_deltax_square = 0
    i = 0
    while i < len(delta_x):
        sum_deltax_deltay += (delta_x[i] * delta_y[i])
        sum_deltax_square += (delta_x[i] ** 2)
        i+=1
    return float(sum_deltax_deltay/sum_deltax_square)

def get_y_intercept_and_slope(xList, yList):
    m = calc_slope(xList, yList)
    b = get_mean(yList) - m * get_mean(xList)
    return {'m': m, 'b': b}

We have one issue here; some of the variables e.g. `sex`, `smoker` and `region` don't have numerical values therefore they cannot be plotted on numerical x-axis. To resolve this issue we have to convert these variables values to numerical equivalents. 

For this purpose we need to come up with a function that finds all the unique values of the variable and allot a numerical value to the string vazlues. The unique values will be the **keys** of the dictionary and the numerical values will be the **values** of the dictionary

In [6]:
def get_unique_values_dict(list):
    dict = {}
    unique_value = 0
    for i in list:
        if i not in dict:
            dict[i] = unique_value
            unique_value+=1
    return dict

def get_numerical_values(list, dict):
    converted = []
    for i in list:
        converted.append(dict[i])
    return converted

Next we are going to find the `slope` and `y-intercept` between `charges` and rest of the variables i.e. `age`, `sex`, `bmi`, `children`, `smoker` and `region`. For this puropse we will keep the `charges` variable on y-axis while the others on x-axis one by one.

In [7]:
processed_data = {}
y_axis = variables['charges']

for k,v in variables.items():
    if k != 'charges':
        x_axis = []
        if k == 'sex' or k == 'smoker' or k == 'region':
            x_axis = get_numerical_values(v, get_unique_values_dict(v))
        else:
            x_axis = v
        if k not in processed_data:
            processed_data[k] = {
                                    'slope_and_yIntercept': get_y_intercept_and_slope(x_axis, y_axis),
                                    'x_axis': x_axis
                                }      
            
#print(processed_data)

## Calculate Error

At this point the processed_data contains the `slope` and the `y-intercept` of all the variables with respect to the `charges`. Next we need to calculate the error or distance of data points from the best-fitting line. 

For this we will be calculating the **y** using **x**, **slope** and **y-intercept**.

Then the absolute difference of actual y and the y calculated using linear regression equation will be the error.

The sum of errors of all the points will tell us about how weak or strong the relationship is between variables on x-axis and y-axis. The higher the error the weaker will be the relationship.

In [8]:
def get_y(m, b, x):
    return float(float(m)*float(x) + b)

def calculate_error(m, b, point):
    x = point[0]
    y = point[1]
    y_value = get_y(m, b, x)
    return abs(float(y)-y_value)

def calculate_all_error(m, b, points):
    error = 0;
    for p in points:
        error += calculate_error(m, b, p)
    return error

for k,v in processed_data.items():
    points = []
    m = v['slope_and_yIntercept']['m']
    b = v['slope_and_yIntercept']['b']
    i = 0
    while i<len(y_axis):
        point = [v['x_axis'][i], y_axis[i]]
        points.append(point)
        i+=1
    processed_data[k]["Error"] = calculate_all_error(m, b, points)
    processed_data[k].pop('x_axis')
    
print(processed_data)


{'age': {'slope_and_yIntercept': {'m': 258.39963740975935, 'b': 3132.732296926866}, 'Error': 12103198.168465717}, 'sex': {'slope_and_yIntercept': {'m': -1393.7008439563995, 'b': 13956.7511777219}, 'Error': 12154917.530519988}, 'bmi': {'slope_and_yIntercept': {'m': 394.1346504088604, 'b': 1181.3969985225085}, 'Error': 12266477.379487379}, 'children': {'slope_and_yIntercept': {'m': 685.5511224819064, 'b': 12516.535282216162}, 'Error': 12166754.928700505}, 'smoker': {'slope_and_yIntercept': {'m': 23671.514111814657, 'b': 8434.268297856095}, 'Error': 7559210.723859744}, 'region': {'slope_and_yIntercept': {'m': -641.3158786479939, 'b': 14200.672738017463}, 'Error': 12128981.890660578}}


## Find Relationship Strength In Descending Order

In [20]:
strongly_related = []

i = 0
while i < len(processed_data):
    min_error = float('inf')
    strongly_related.append('')
    for k,v in processed_data.items():
        if k in strongly_related:
            continue
        else:
            if v['Error'] < min_error:
                min_error = v['Error']
                strongly_related[i] = k
    i+=1
    
print(strongly_related)
    
    

['smoker', 'age', 'region', 'sex', 'children', 'bmi']


Now we have the strength of relationship of variables with `charges` of the insurance.

The linear regression analysis reveals the relationship strength in the following descending order.

1. `smoker`
2. `age`
3. `region`
4. `sex`
5. `children`
6. `bmi`

Hance being a `smoker` has the highest impact on the cost of insurance while `bmi` has the least impact on the  cost of insurance.


In [70]:
insurance_dict = {}
i=0;
with open('Data/insurance.csv') as insurance_csv:
    insurance_dict_reader = csv.DictReader(insurance_csv)
    for row in insurance_dict_reader:
        insurance_dict[i] = row
        i+=1

for smoker_vs_charges in sorted(insurance_dict.values(), key = lambda item: float(item['charges'])):
    #print(smoker_vs_charges)
    print("{}\t{}".format(smoker_vs_charges['smoker'], smoker_vs_charges['charges']))

no	1121.8739
no	1131.5066
no	1135.9407
no	1136.3994
no	1137.011
no	1137.4697
no	1141.4451
no	1146.7966
no	1149.3959
no	1163.4627
no	1241.565
no	1242.26
no	1242.816
no	1252.407
no	1253.936
no	1256.299
no	1261.442
no	1261.859
no	1263.249
no	1391.5287
no	1515.3449
no	1526.312
no	1532.4697
no	1534.3045
no	1607.5101
no	1615.7667
no	1621.3402
no	1621.8827
no	1622.1885
no	1625.43375
no	1627.28245
no	1628.4709
no	1629.8335
no	1631.6683
no	1631.8212
no	1632.03625
no	1632.56445
no	1633.0444
no	1633.9618
no	1634.5734
no	1635.73365
no	1639.5631
no	1639.5631
no	1646.4297
no	1664.9996
no	1674.6323
no	1682.597
no	1694.7964
no	1702.4553
no	1704.5681
no	1704.70015
no	1705.6245
no	1708.0014
no	1708.92575
no	1711.0268
no	1712.227
no	1719.4363
no	1720.3537
no	1725.5523
no	1727.54
no	1727.785
no	1728.897
no	1731.677
no	1737.376
no	1743.214
no	1744.465
no	1748.774
no	1759.338
no	1769.53165
no	1815.8759
no	1824.2854
no	1826.843
no	1832.094
no	1837.237
no	1837.2819
no	1842.519
no	1875.344
no	1877.9294
no	1880

## Evaluate Data Bias

In [93]:
import statistics as stat
import numpy as np

data_stats = {}

for k,v in variables.items():
    if k != 'charges' and k != 'sex' and k != 'smoker' and k != 'region' and k != 'id':
        x_axis = [float(x) for x in v]
        if k not in data_stats:
            data = np.array(x_axis)
            q3, q1 = np.percentile(data, [75 ,25])
            data_stats[k] = {
                                'mean': round(stat.mean(x_axis),2),
                                'mode': round(stat.mode(x_axis),2),
                                'std': round(stat.stdev(x_axis),2),
                                'median': round(stat.median(x_axis),2),
                                'iqr': round(q3 - q1,2),
                                'median_diff_max': round(max(x_axis) - stat.median(x_axis),2),
                                'median_diff_min': round(stat.median(x_axis) - min(x_axis),2)
                            }

print("{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}".format('var', 'mean', 'mode', 'std', 'median', 'iqr', 'med_diff_max', 'med_diff_min'))            
for k,v in data_stats.items():
    print("{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}\t\t{}".format(k, v['mean'], v['mode'], v['std'], v['median'], v['iqr'], v['median_diff_max'], v['median_diff_min']))

var		mean		mode		std		median		iqr		med_diff_max		med_diff_min
age		39.22		18.0		14.04		39.0		24.0		25.0		21.0
bmi		30.67		32.3		6.1		30.4		8.41		22.73		14.44
children		1.1		0.0		1.21		1.0		2.0		4.0		1.0


We can observe  here that there is no much difference metween the **standard daviation** and **IQR** as well as there is no much difference between the **mean** and the **median** for `age`, `bmi` and `children` variables. Therefore there is no skewness in the distribution and hance outliers are minimal.

The **standard daviation** in `age` is a bit high which is good. This indicates that the data has variation of age groups.

The **standard daviation** in `bmi` and `children` is very low, This indicates that the data for bmi and children do not have variation.